# L'approssimazione di una funzione lineare mediante un singolo neurone a comportamento lineare

Luca Mari, settembre 2024  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere, a partire da un esempio concreto, che una rete neurale deve includere degli elementi non lineari per poter approssimare appropriatamente anche delle semplici funzioni non lineari.  
**Precompetenze**: basi di Python; almeno qualche idea di analisi matematica.

> Per eseguire questo notebook, supponiamo con VSCode, occorre:
> * installare un interprete Python
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [neuron.ipynb](neuron.ipynb)
>     * aprire il notebook `neuron.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare il modulo Python richiesto, eseguendo dal terminale:  
>         `pip install torch`

Una rete neurale è l'implementazione di una funzione parametrica $Y = f(X; \theta)$, e può essere intesa come uno strumento di approssimazione di funzioni $F(X)$ date: attraverso un opportuno addestramento, si trovano i valori appropriati dei parametri $\theta$ in modo che $f(X; \theta) \approx F(X)$.

Quest'idea venne sviluppata inizialmente assumendo che i componenti elementari di una rete -- i suoi neuroni -- avessero un comportamento lineare:  

![rete](neuron.drawio.svg)  
nel caso di due input.

La situazione più semplice è ovviamente quella di una rete costituita da un solo neurone. Facciamo qualche prova.

Per costruire e operare sulla rete useremo `PyTorch`: importiamo perciò i moduli Python che saranno necessari e verifichiamo se è disponibile una GPU per eseguire la rete.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Costruiamo la rete usando `PyTorch` (il codice ha un po' di dettagli tecnici, non necessariamente importanti: i commenti potrebbero essere comunque utili) e visualizziamo i valori dei suoi parametri, che inizialmente sono casuali.

In [2]:
class OneNeuron(nn.Module):
    def __init__(self, device):
        super(OneNeuron, self).__init__()
        self.neuron = nn.Linear(2, 1)

        self.loss = nn.MSELoss()        # funzione di errore: Mean Squared Error
        self.optimizer = optim.SGD(self.parameters(), lr=0.01) # ottimizzatore: Stochastic Gradient Descent
        self.device = device
        self.to(device)

    def forward(self, x):
        x = self.neuron(x)
        return x

    def set_learning_rate(self, learning_rate):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = learning_rate        

    def train(self, x, y, epochs, repeat):
        print(f'\n*** Addestramento ***\nepoca\terrore (su {device})')
        for epoch in range(epochs):
            self.optimizer.zero_grad()  # azzera i gradienti
            output = self(x)            # calcola l'output
            loss = self.loss(output, y) # calcola la funzione di errore
            loss.backward()             # calcola i gradienti
            self.optimizer.step()       # aggiorna i valori dei parametri
            if (epoch+1) % repeat == 0:
                print(f'{epoch+1}\t{loss.item():.3f}')

    def predict(self, examples, fun):
        print('\n*** Inferenza ***')
        x_test = examples
        y_test = self(x_test)           # calcola la previsione
        y_true = self.calc_fun(fun, x_test)
        print('x1\tx2\ty\ty prev\terrore')
        for i in range(x_test.size(0)):
            x1, x2 = x_test[i][0].item(), x_test[i][1].item()
            y, y_hat = y_true[i].item(), y_test[i].item()
            print(f'{x1:.2f}\t{x2:.2f}\t{y:.2f}\t{y_hat:.2f}\t{y - y_hat:.2f}')
        print(f'Errore quadratico medio: {torch.mean((y_test - y_true)**2):.5f}')

    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

    def print_parameters(self):
        for name, param in self.named_parameters():
            print(name, param.data)

    def calc_fun(self, fun, X):
        return fun(X[:, 0], X[:, 1]).view(-1, 1).to(self.device)


model = OneNeuron(device)
print('I parametri della rete sono:'); model.print_parameters()

I parametri della rete sono:
neuron.weight tensor([[-0.5119,  0.2495]], device='cuda:0')
neuron.bias tensor([-0.1720], device='cuda:0')


Costruiamo il training set, prima di tutto negli input (_features_, _covariates_) come un certo numero di coppie di numeri casuali.

In [3]:
def examples(n): return (10 * torch.rand(n, 2) - 5).to(device) # genera n esempi nella forma ognuno di una coppia di numeri casuali tra -5 e 5
num_examples = 100                      # numero di esempi per il training set
X = examples(num_examples)              # calcola i valori degli esempi: input del training set

Scegliamo la funzione, dunque a due argomenti, da approssimare. Essendo un caso di _supervised learning_, calcoliamo la funzione per tutte le coppie del training set e aggiungiamo il risultato al training set stesso.

In [4]:
def fun(x1, x2): return (x1 + x2) / 2   # funzione da approssimare, in questo caso la media tra due numeri
Y = model.calc_fun(fun, X)              # calcola il valore della funzione per ogni esempio: output del training set

Già ora possiamo mettere in funzione la rete, su un certo numero di esempi che costituiscono dunque un test set, ma ovviamente il risultato non sarà in alcun modo accurato.

In [5]:
model.predict(examples(10), fun)        # inferenza prima dell'addestramento


*** Inferenza ***
x1	x2	y	y prev	errore
2.65	-0.36	1.14	-1.62	2.76
-4.98	3.88	-0.55	3.34	-3.89
2.72	-3.83	-0.56	-2.52	1.96
0.47	1.30	0.89	-0.09	0.97
-2.23	-2.57	-2.40	0.33	-2.73
2.97	4.53	3.75	-0.56	4.31
3.15	3.71	3.43	-0.86	4.29
-1.08	3.34	1.13	1.21	-0.08
-3.41	0.94	-1.24	1.81	-3.05
-4.87	3.07	-0.90	3.09	-3.99
Errore quadratico medio: 9.72123


Addestriamo allora la rete, dopo aver assegnato valori opportuni ai due iperparametri fondamentali:  
-- il numero di volte in cui il processo di addestramento viene ripetuto, e  
-- la velocità di apprendimento (_learning rate_).

In [6]:
num_epochs = 100                        # numero di ripetizioni del processo di addestramento
repeat = 10                             # numero di ripetizioni dopo le quali visualizzare l'errore
model.reset_parameters()                # reinizializza i parametri della rete
model.set_learning_rate(0.02)           # imposta il learning rate
model.train(X, Y, num_epochs, repeat)   # addestra la rete


*** Addestramento ***
epoca	errore (su cuda)
10	0.195
20	0.088
30	0.040
40	0.018
50	0.008
60	0.004
70	0.002
80	0.001
90	0.000
100	0.000


Mettiamo in funzione la rete su un nuovo test set: se l'addestramento ha avuto successo, si dovrebbe ottenere un piccolo errore quadratico medio.

In [7]:
model.predict(examples(10), fun)        # inferenza dopo l'addestramento


*** Inferenza ***
x1	x2	y	y prev	errore
2.40	2.12	2.26	2.27	-0.01
3.92	3.81	3.87	3.87	-0.01
2.31	2.16	2.23	2.24	-0.01
-0.82	-4.13	-2.47	-2.46	-0.01
-0.33	3.95	1.81	1.82	-0.01
-4.64	-0.13	-2.38	-2.37	-0.01
-3.99	-2.20	-3.10	-3.08	-0.02
-0.87	0.81	-0.03	-0.02	-0.01
-0.17	-4.81	-2.49	-2.47	-0.01
1.99	3.77	2.88	2.89	-0.01
Errore quadratico medio: 0.00014


Visualizziamo i valori dei parametri della rete: se l'addestramento ha avuto successo, dovrebbero essere vicini ai valori attesi.

In [8]:
model.print_parameters()

neuron.weight tensor([[0.4995, 0.4995]], device='cuda:0')
neuron.bias tensor([0.0120], device='cuda:0')


D'altra parte, è evidente che un singolo neurone a comportamento lineare può approssimare efficacemente solo funzioni molto semplici. Anche aumentando il numero di esempi e di ripetizioni del processo di addestramento, per esempio non è in grado di approssimare in modo accettabile la funzione massimo tra due numeri.

In [9]:
num_examples = 1000                     # numero di esempi per il training set
X = examples(num_examples)              # input del training set
def fun(x1, x2): return torch.max(x1, x2) # funzione da approssimare, in questo caso il massimo tra due numeri
Y = model.calc_fun(fun, X)              # calcola il valore della funzione per ogni esempio: output del training set
num_epochs = 1000                       # numero di ripetizioni del processo di addestramento
repeat = 100                            # numero di ripetizioni dopo le quali visualizzare l'errore
model.reset_parameters()                # reinizializza i parametri della rete
model.set_learning_rate(0.01)           # imposta il learning rate
model.train(X, Y, num_epochs, repeat)   # addestra la rete
model.predict(examples(10), fun)        # metti in funzione la rete


*** Addestramento ***
epoca	errore (su cuda)
100	1.515
200	1.452
300	1.451
400	1.451
500	1.451
600	1.451
700	1.451
800	1.451
900	1.451
1000	1.451

*** Inferenza ***
x1	x2	y	y prev	errore
-0.74	4.40	4.40	3.48	0.92
-2.14	-0.06	-0.06	0.53	-0.58
-4.78	0.56	0.56	-0.53	1.09
0.21	-2.42	0.21	0.56	-0.35
3.96	0.50	3.96	3.96	-0.00
-3.69	2.05	2.05	0.77	1.27
3.48	3.47	3.48	5.19	-1.71
3.68	3.55	3.68	5.34	-1.65
-4.85	2.13	2.13	0.22	1.92
-0.95	-3.76	-0.95	-0.71	-0.24
Errore quadratico medio: 1.35233


Per ottenere approssimazioni accettabili occorre dunque costruire una rete più complessa.